In [1]:
#ssa_life_expectancy_sex_year
#life_expectancy_sex_state
#life_expectancy_state_county
#life_expectancy_race_sex
#life_expectancy_race_sex_year

In [2]:
import pandas as pd
import sqlite3

In [3]:
db_file = "life_expectancy_tables.db"

In [4]:
df0 = pd.read_csv('133_life-expectancy_reformatted.csv')

In [5]:
print(df0.head(3))

   Year       Race         Sex  LifeExpectancy
0  1940  All Races  Both Sexes            62.9
1  1940  All Races      Female            65.2
2  1940  All Races        Male            60.8


In [7]:
table_life_expectancy_race_sex_year = "life_expectancy_race_sex_year"
conn = sqlite3.connect(db_file)
df0.to_sql(table_life_expectancy_race_sex_year, conn, if_exists="replace", index=False)
conn.close()

In [3]:
# Load the two CSV files into pandas DataFrames
df1 = pd.read_csv('SSA_Life_Expectancy_by_sex_1940_2001.csv')
df2 = pd.read_csv('SSA_Life_Expectancy_by_sex_2002_2080.csv')

# Concatenate the DataFrames vertically (appending rows)
# Since column names are the same, pandas will align them automatically.
# The 'ignore_index=True' argument ensures a continuous index in the combined DataFrame.
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a new CSV file
# The 'index=False' argument prevents pandas from writing the DataFrame index as a column in the CSV.
#combined_df.to_csv('SSA_Life_Expectancy_by_sex_1940_2080.csv', index=False)

In [4]:
# Reshape from wide to long format
df_long = combined_df.melt(
    id_vars=["Year"],          # keep Year as identifier
    value_vars=["Male", "Female"],  # columns to unpivot
    var_name="Sex",            # new column name for 'Male'/'Female'
    value_name="LifeExpectancy" # new column name for values
)

# Save to a new CSV
df_long.to_csv("SSA_Life_Expectancy_by_sex_1940_2080.csv", index=False)

print(df_long.head(6))


   Year   Sex  LifeExpectancy
0  1940  Male            61.4
1  1941  Male            61.9
2  1942  Male            62.6
3  1943  Male            62.3
4  1944  Male            62.7
5  1945  Male            62.9


In [10]:
#table_ssa = "ssa_life_expectancy_sex_year"
#conn = sqlite3.connect(db_file)
#df_long.to_sql(table_ssa, conn, if_exists="replace", index=False)
#conn.close()

In [6]:
df3 = pd.read_csv('U.S._State_Life_Expectancy_by_Sex__2020.csv')
print(df3.head(6))

        State         Sex  LifeExpectancy  StandardError  \
0     Alabama  Both Sexes            73.2          0.067   
1      Alaska  Both Sexes            76.6          0.176   
2     Arizona  Both Sexes            76.3          0.055   
3    Arkansas  Both Sexes            73.8          0.086   
4  California  Both Sexes            79.0          0.022   
5    Colorado  Both Sexes            78.3          0.058   

   LifeExpectancyQuartileLow  LifeExpectancyQuartileHigh  
0                       71.9                        75.3  
1                       75.4                        76.8  
2                       75.4                        76.8  
3                       71.9                        75.3  
4                       78.1                        80.7  
5                       78.1                        80.7  


In [7]:
df3a = df3.drop(columns=['StandardError','LifeExpectancyQuartileLow','LifeExpectancyQuartileHigh'])
df3a.head()

,State,Sex,LifeExpectancy
0,Alabama,Both Sexes,73.2
1,Alaska,Both Sexes,76.6
2,Arizona,Both Sexes,76.3
3,Arkansas,Both Sexes,73.8
4,California,Both Sexes,79.0


In [11]:
table_less = "life_expectancy_sex_state"
conn = sqlite3.connect(db_file)
df3a.to_sql(table_less, conn, if_exists="replace", index=False)
conn.close()

In [12]:
df4a = pd.read_csv('U.S._Life_Expectancy_at_Birth_by_State_and_Census_Tract_-_2010-2015.csv')
print(df4a.head(6))

     State              County  CensusTract  LifeExpectancy  \
0  Alabama                 NaN          NaN            75.5   
1  Alabama  Autauga County, AL        201.0            73.1   
2  Alabama  Autauga County, AL        202.0            76.9   
3  Alabama  Autauga County, AL        203.0             NaN   
4  Alabama  Autauga County, AL        204.0            75.4   
5  Alabama  Autauga County, AL        205.0            79.4   

  LifeExpectancyRange  LifeExpectancyStandardError  
0           75.2-77.5                       0.0328  
1           56.9-75.1                       2.2348  
2           75.2-77.5                       3.3453  
3                 NaN                          NaN  
4           75.2-77.5                       1.0216  
5           77.6-79.5                       1.1768  


In [13]:
df4 = df4a.drop(columns=['CensusTract','LifeExpectancyRange','LifeExpectancyStandardError'])
print(df4.head(6))

     State              County  LifeExpectancy
0  Alabama                 NaN            75.5
1  Alabama  Autauga County, AL            73.1
2  Alabama  Autauga County, AL            76.9
3  Alabama  Autauga County, AL             NaN
4  Alabama  Autauga County, AL            75.4
5  Alabama  Autauga County, AL            79.4


In [15]:
df4['County'] = df4['County'].str.split(',').str[0]
df4a = df4.dropna(subset=['County'])
print(df4a.head(5))

     State          County  LifeExpectancy
1  Alabama  Autauga County            73.1
2  Alabama  Autauga County            76.9
3  Alabama  Autauga County             NaN
4  Alabama  Autauga County            75.4
5  Alabama  Autauga County            79.4


In [17]:
df4 = (
    df4a.groupby(["State", "County"], as_index=False)
      .agg({
          "LifeExpectancy": "mean",
      })
)
print(df4.head(10))

     State           County  LifeExpectancy
0  Alabama   Autauga County       75.320000
1  Alabama   Baldwin County       77.482759
2  Alabama   Barbour County       73.971429
3  Alabama      Bibb County       73.550000
4  Alabama    Blount County       76.077778
5  Alabama   Bullock County       77.600000
6  Alabama    Butler County       73.387500
7  Alabama   Calhoun County       73.136000
8  Alabama  Chambers County       73.611111
9  Alabama  Cherokee County       74.083333


In [18]:
table_lesc = "life_expectancy_state_county"
conn = sqlite3.connect(db_file)
df4.to_sql(table_lesc, conn, if_exists="replace", index=False)
conn.close()

In [20]:
df5 = pd.read_csv('NCHS_-_Death_rates_and_life_expectancy_at_birth.csv')

In [21]:
print(df5.head(5))

   Year       Race         Sex  LifeExpectancy  AgeAdjustedDeathRate
0  1900  All Races  Both Sexes            47.3                2518.0
1  1901  All Races  Both Sexes            49.1                2473.1
2  1902  All Races  Both Sexes            51.5                2301.3
3  1903  All Races  Both Sexes            50.5                2379.0
4  1904  All Races  Both Sexes            47.6                2502.5


In [23]:
df5a = df5.drop(columns=['AgeAdjustedDeathRate'])
print(df5a.head())

   Year       Race         Sex  LifeExpectancy
0  1900  All Races  Both Sexes            47.3
1  1901  All Races  Both Sexes            49.1
2  1902  All Races  Both Sexes            51.5
3  1903  All Races  Both Sexes            50.5
4  1904  All Races  Both Sexes            47.6


In [24]:
table_race = "life_expectancy_race_sex"
conn = sqlite3.connect(db_file)
df5a.to_sql(table_race, conn, if_exists="replace", index=False)
conn.close()